In [1]:
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
#data.x: Node feature matrix with shape [num_nodes, num_node_features]
#data.edge_index: Graph connectivity in COO format with shape [2, num_edges] and type torch.long
#data.edge_attr: Edge feature matrix with shape [num_edges, num_edge_features]
#data.y: Target to train against (may have arbitrary shape), e.g., node-level targets of shape [num_nodes, *] or graph-level targets of shape [1, *]
#data.pos: Node position matrix with shape [num_nodes, num_dimensions]

edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index, pos= edge_atrr=, y=)
print(data)

Data(x=[3, 1], edge_index=[2, 4])


In [ ]:
# if edge_index is of the form [num_edges, 2] then call edge_index.transpose().contiguous()
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())
#loader = DataLoader([groah1, graph2, graoh3, ...], batch_size=32, shuffle=True)
#for batch in loader: model(batch)
print(data)

Data(x=[3, 1], edge_index=[2, 4])


In [5]:
dataset = TUDataset(root="TUDatasets", name="ENZYMES")
print(type(dataset), type(dataset[0]), dataset[0])
print(dataset.num_node_features)
print(dataset.num_classes)

<class 'torch_geometric.datasets.tu_dataset.TUDataset'> <class 'torch_geometric.data.data.Data'> Data(edge_index=[2, 168], x=[37, 3], y=[1])
3
6


In [12]:
loader = DataLoader(dataset, batch_size=32, shuffle=True)
print(len(loader.dataset))
for batch_nr, batch_dat  in enumerate(loader):
    print(batch_nr, type(batch_dat), batch_dat.y, len(batch_dat))

600
0 <class 'abc.DataBatch'> tensor([5, 5, 5, 3, 3, 4, 2, 3, 1, 0, 2, 3, 4, 1, 2, 2, 4, 2, 4, 1, 2, 3, 3, 4,
        3, 2, 5, 1, 4, 2, 0, 0]) 32
1 <class 'abc.DataBatch'> tensor([1, 2, 3, 5, 4, 3, 1, 0, 2, 1, 4, 1, 4, 0, 1, 3, 1, 4, 5, 5, 1, 5, 3, 3,
        1, 5, 3, 5, 3, 0, 3, 1]) 32
2 <class 'abc.DataBatch'> tensor([2, 5, 4, 0, 3, 2, 0, 0, 3, 0, 1, 1, 5, 2, 4, 2, 4, 1, 0, 4, 1, 4, 2, 1,
        2, 1, 2, 3, 1, 5, 4, 4]) 32
3 <class 'abc.DataBatch'> tensor([0, 2, 3, 3, 0, 1, 5, 5, 5, 1, 4, 1, 3, 5, 3, 2, 0, 3, 4, 1, 4, 5, 0, 3,
        4, 1, 5, 0, 1, 3, 0, 0]) 32
4 <class 'abc.DataBatch'> tensor([5, 0, 0, 4, 3, 4, 3, 2, 5, 1, 0, 5, 5, 2, 0, 5, 4, 1, 2, 2, 3, 0, 1, 3,
        2, 0, 1, 1, 4, 3, 1, 2]) 32
5 <class 'abc.DataBatch'> tensor([3, 1, 4, 3, 5, 0, 1, 1, 5, 1, 0, 1, 0, 2, 4, 1, 2, 1, 3, 4, 1, 5, 3, 0,
        4, 0, 4, 5, 1, 3, 2, 2]) 32
6 <class 'abc.DataBatch'> tensor([1, 0, 0, 1, 3, 1, 4, 5, 4, 4, 4, 0, 1, 0, 5, 3, 2, 3, 1, 4, 0, 1, 4, 3,
        0, 0, 3, 1, 5, 2, 0, 5]) 32
7 

In [49]:
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super().__init__()
        self.conv1 = torch_geometric.nn.GCNConv(num_node_features, 16)
        self.conv2 = torch_geometric.nn.GCNConv(16, 10)
        self.conv3 = torch_geometric.nn.GCNConv(10, num_classes)

    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.nn.functional.relu(x)
        #x = torch.nn.functional.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = torch.nn.functional.relu(x)

        x = self.conv3(x, edge_index)

        return torch.nn.functional.log_softmax(x, dim=1)

model = GCN(dataset.num_node_features, dataset.num_classes).to(device)